# Введение в искусственные нейронные сети
# Урок 4. Сверточные нейронные сети

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D 

0 футболка/клубка
1 Брюки
2 Пуловер
3 Платье
4 Пальто
5 Сандал
6 Рубашка
7 Кроссовок
8 Сумка
9 Ботинок

In [2]:
# загрузка тренировочных и тестовых данных
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# конвертация чисел из uint8 в float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255 
x_test /= 255 

# трансформация лейблов в one-hot encoding
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

# изменение размерности массива в 4D массив
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

4423680/4422102 [==============================] - 0s 0us/step


Рассмотрим сеть с разным количеством ядер

In [15]:
df = pd.DataFrame(columns = ['n Conv2D', 'n pool', 'n Dense', 'k1', 'k2', 'k3', 'test loss', 'accuracy'])

In [ ]:
k1_n = [6, 24, 100]
k2_n = [8, 32, 100]
k3_n = [30, 60, 120]
k = 1
for k1 in k1_n:
  for k2 in k2_n:
    for k3 in k3_n:
      n_c = 0
      n_p = 0
      n_d = 0
      model = Sequential()
      # первый сверточный слой 6 - количество ядер
      model.add(layers.Conv2D(k1, kernel_size=(15, 15), strides=(1, 1), activation='tanh', input_shape=(28,28,1), padding="same"))
      n_c += 1
      # второй пуллинговый слой
      model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
      n_p+=1
      # третий сверточный слой
      model.add(layers.Conv2D(k2, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      # четвертый пуллинговый слой
      model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
      n_p += 1
      # пятый полносвязный слой
      model.add(layers.Conv2D(k3, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
      model.add(layers.Flatten())
      # шестой полносвязный слой
      model.add(layers.Dense(84, activation='tanh'))
      n_d += 1
      # выходной слой с функцией активации softmax
      model.add(layers.Dense(10, activation='softmax'))
      # компилияция модели
      model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])
      hist = model.fit(x=x_train,y=y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), verbose=1)
      test_score = model.evaluate(x_test, y_test)
      
      print(f'k1= {k1}, k2= {k2}, k3= {k3}')
      print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))
      k += 1
      df.loc[k] = [n_c, n_p, n_d, k1, k2, k3, test_score[0], test_score[1]]

In [17]:
df[df['accuracy']>0.8]

,n Conv2D,n pool,n Dense,k1,k2,k3,test loss,accuracy
2,3.0,2.0,1.0,6.0,8.0,30.0,0.491722,0.8201
3,3.0,2.0,1.0,6.0,8.0,60.0,0.476409,0.8296
4,3.0,2.0,1.0,6.0,8.0,120.0,0.511865,0.8147
5,3.0,2.0,1.0,6.0,32.0,30.0,0.479576,0.8279
6,3.0,2.0,1.0,6.0,32.0,60.0,0.472246,0.8321
7,3.0,2.0,1.0,6.0,32.0,120.0,0.465404,0.8343
8,3.0,2.0,1.0,6.0,100.0,30.0,0.473696,0.8304
9,3.0,2.0,1.0,6.0,100.0,60.0,0.463715,0.8335
10,3.0,2.0,1.0,6.0,100.0,120.0,0.464610,0.8319
11,3.0,2.0,1.0,24.0,8.0,30.0,0.483854,0.8235


Видим, что при увеличении количества ядер точность увеличивается, правда при больших значения количества ядер точность увеличивается на малые величины, а иногда даже падает

In [18]:
df[df['accuracy']>0.835]
#лучший результат - оставим 24, 100 - 100 -60

,n Conv2D,n pool,n Dense,k1,k2,k3,test loss,accuracy
15,3.0,2.0,1.0,24.0,32.0,60.0,0.456584,0.8380
18,3.0,2.0,1.0,24.0,100.0,60.0,0.450268,0.8399
23,3.0,2.0,1.0,100.0,32.0,30.0,0.465260,0.8360
26,3.0,2.0,1.0,100.0,100.0,30.0,0.457276,0.8369
27,3.0,2.0,1.0,100.0,100.0,60.0,0.448560,0.8391


Теперь будем увеличивать количество слоев

In [4]:
df1 = pd.DataFrame(columns = ['n Conv2D', 'n pool', 'n Dense', 'k1', 'k2', 'k3', 'test loss', 'accuracy'])

In [17]:
k1_n = [24, 100]
k2_n = [100]
k3_n = [60]
#k = 4
for k1 in k1_n:
  for k2 in k2_n:
    for k3 in k3_n:
      n_c = 0
      n_p = 0
      n_d = 0
      model = Sequential()
      # первый сверточный слой 6 - количество ядер
      model.add(layers.Conv2D(k1, kernel_size=(15, 15), strides=(1, 1), activation='tanh', input_shape=(28,28,1), padding="same"))
      n_c += 1
      # второй пуллинговый слой
      model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
      n_p+=1
      model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
      n_p+=1
      # третий сверточный слой
      model.add(layers.Conv2D(k2, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      model.add(layers.Conv2D(k2, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      #add
      model.add(layers.Conv2D(k2, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      # четвертый пуллинговый слой
      model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
      n_p += 1
      # пятый полносвязный слой
      model.add(layers.Conv2D(k3, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      #add
      model.add(layers.Conv2D(k3, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding='valid'))
      n_c += 1
      # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
      model.add(layers.Flatten())
      # шестой полносвязный слой
      model.add(layers.Dense(84, activation='tanh'))
      n_d += 1
      #add
      model.add(layers.Dense(84, activation='tanh'))
      n_d += 1
      model.add(layers.Dense(84, activation='tanh'))
      n_d += 1
      model.add(layers.Dense(84, activation='tanh'))
      n_d += 1
      model.add(layers.Dense(4, activation='tanh'))
      n_d += 1
      # выходной слой с функцией активации softmax
      model.add(layers.Dense(10, activation='softmax'))
      # компилияция модели
      model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])
      hist = model.fit(x=x_train,y=y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), verbose=1)
      test_score = model.evaluate(x_test, y_test)
      
      print(f'k1= {k1}, k2= {k2}, k3= {k3}')
      print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))
      k += 1
      df1.loc[k] = [n_c, n_p, n_d, k1, k2, k3, test_score[0], test_score[1]]

Epoch 1/5
469/469 [==============================] - 12s 25ms/step - loss: 1.7193 - accuracy: 0.4918 - val_loss: 1.4593 - val_accuracy: 0.5672
Epoch 2/5
469/469 [==============================] - 12s 25ms/step - loss: 1.3258 - accuracy: 0.5960 - val_loss: 1.2302 - val_accuracy: 0.6276
Epoch 3/5
469/469 [==============================] - 12s 25ms/step - loss: 1.1295 - accuracy: 0.6564 - val_loss: 1.0502 - val_accuracy: 0.7122
Epoch 4/5
469/469 [==============================] - 12s 25ms/step - loss: 0.9612 - accuracy: 0.7314 - val_loss: 0.9055 - val_accuracy: 0.7327
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.8232 - accuracy: 0.7405
k1= 24, k2= 100, k3= 60
Test loss 0.8232, accuracy 74.05%
Epoch 1/5
469/469 [==============================] - 17s 37ms/step - loss: 1.6527 - accuracy: 0.4773 - val_loss: 1.4397 - val_accuracy: 0.5598
Epoch 2/5
469/469 [==============================] - 17s 36ms/step - loss: 1.3043 - accuracy: 0.6600 - val_loss: 1.2215 - val_ac

In [18]:
df1

,n Conv2D,n pool,n Dense,k1,k2,k3,test loss,accuracy
2,5.0,2.0,1.0,24.0,100.0,60.0,0.475311,0.8288
3,5.0,2.0,1.0,100.0,100.0,60.0,0.456797,0.8355
5,5.0,2.0,2.0,24.0,100.0,60.0,0.466031,0.8353
6,5.0,2.0,2.0,100.0,100.0,60.0,0.467143,0.8348
7,5.0,3.0,3.0,24.0,100.0,60.0,0.452069,0.8338
8,5.0,3.0,3.0,100.0,100.0,60.0,0.454655,0.8408
9,6.0,3.0,4.0,24.0,100.0,60.0,0.446590,0.8377
10,6.0,3.0,4.0,100.0,100.0,60.0,0.458205,0.8352
11,6.0,3.0,5.0,24.0,100.0,60.0,0.879693,0.7562
12,6.0,3.0,5.0,100.0,100.0,60.0,0.846360,0.7759


Видим, что с увеличением слоев увеличивается точность, но потом точность падает, видимо наступает переобучение
лучший результат - 5 слоев свертки, 3 слоя пулинга, 3 полносвязных слоя 

## Практическое задание

Вариант 1. (простой)

- обучить сверточную нейронную сеть на датасете fashion-mnist
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
    </li>

Вариант 2. (сложный)
- реализовать нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на сifar10.
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
    </li>
</ol>

## Дополнительные материалы

<ol>
    <li>https://keras.io/layers/convolutional/</li>
    <li>https://keras.io/layers/pooling/</li>
    <li>https://keras.io/preprocessing/image/</li>
</ol>

## Используемая литература 

Для подготовки данного методического пособия были использованы следующие ресурсы:
<ol>
    <li>https://keras.io</li>
    <li>Шакла Н. — Машинное обучение и TensorFlow 2019</li>
    <li>Николенко Сергей Игоревич, Кадурин А. А. - Глубокое обучение. Погружение в мир нейронных сетей  2018</li>
    <li>Francois Chollet - Deep Learning with Python 2018</li>
    <li>Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton - ImageNet Classification with Deep Convolutional Neural Networks</li>
    <li>Karen Simonyan, Andrew Zisserman - Very Deep Convolutional Networks for Large-Scale Image Recognition</li>
    <li>Википедия</li>    
</ol>